In [1]:
import requests
import pandas as pd
import requests 
import os

# Metro mos.ru

***agency.txt***	Обязат.	Транспортные агентства, услуги которых представлены в наборе данных.


***stops.txt***	Обязат.	Остановки, на которых транспортные средства забирают или высаживают пассажиров. Также определяет станции и входы в них.


***routes.txt***	Обязат.	Маршруты общественного транспорта (совокупность рейсов, на которых пассажирам оказывается единая транспортная услуга).


***trips.txt***	Обязат.	Рейсы для каждого маршрута (последовательности из двух или более остановок, на которых транспортное средство останавливается в указанное время).


***stop_times.txt***	Обязат.	Время, когда транспортное средство прибывает на остановки и отправляется от них, выполняя тот или иной рейс.


***calendar.txt***	Условно обязат.	Даты обслуживания, указанные в виде еженедельных периодов с датами начала и окончания. Это поле должно быть заполнено всегда, кроме случаев, когда календарь доступности услуг определен в файле calendar_dates.txt.

## agency.txt

In [23]:
download_url('https://op.mos.ru/EHDWSREST/catalog/export/get?id='+str(847226), temp_file)
df = pd.read_json(temp_file, encoding='cp1251')

In [24]:
agencies = pd.read_csv('data/gtfs/agency.txt')
max_agency_id = agencies.agency_id.max()
max_agency_id

9

In [25]:
pd.concat([
    agencies,
    pd.DataFrame([{'agency_id':max_agency_id+1,
 'agency_name':'ГУП «Московский метрополитен»',
 'agency_url':'http://gup.mosmetro.ru/',
 'agency_timezone':'Europe/Moscow'}])
], sort=False, ignore_index=True).to_csv('data/gtfs/agency.txt', sep=',', index=False)

del agencies
del max_agency_id

## stops.txt

In [26]:
download_url('https://op.mos.ru/EHDWSREST/catalog/export/get?id='+str(847226), temp_file)
df = pd.read_json(temp_file, encoding='cp1251')

In [27]:
stops = pd.read_csv('data/gtfs/stops.txt')
max_stop_id = stops.stop_id.max()
max_stop_id

1009017

In [28]:
df['stop_name'] = df['Name']
df['stop_lon'] = df['Longitude_WGS84']
df['stop_lat'] = df['Latitude_WGS84']

In [29]:
main_stations = df.groupby(['Line', 'NameOfStation']).mean()[['stop_lon', 'stop_lat']].reset_index()
main_stations['location_type'] = 1
main_stations['parent_station'] = None
main_stations = main_stations.rename(columns={'NameOfStation':'stop_name'})
main_stations['stop_id'] = range(max_stop_id+1, max_stop_id+1+len(main_stations))
main_stations.head()

,Line,stop_name,stop_lon,stop_lat,location_type,parent_station,stop_id
0,Арбатско-Покровская линия,Арбатская,37.603279,55.752386,1,None,1009018
1,Арбатско-Покровская линия,Бауманская,37.678778,55.772359,1,None,1009019
2,Арбатско-Покровская линия,Волоколамская,37.382366,55.835529,1,None,1009020
3,Арбатско-Покровская линия,Измайловская,37.781429,55.787719,1,None,1009021
4,Арбатско-Покровская линия,Киевская,37.564620,55.743009,1,None,1009022


In [30]:
df = df.merge(main_stations[['Line',
               'stop_name',
               'stop_id']].rename(columns={'stop_id':'parent_station',
                                          'stop_name':'NameOfStation'}),
        on=['NameOfStation', 'Line'])

In [31]:
max_stop_id = main_stations.stop_id.max()
max_stop_id

1009282

In [32]:
df['stop_id'] = range(max_stop_id+1, max_stop_id+1+len(df))
df['location_type'] = 2

In [33]:
stops = pd.concat([df[['stop_id', 'stop_name', 'stop_lon', 'stop_lat', 'location_type', 'parent_station']],
         main_stations[['stop_id', 'stop_name', 'stop_lon', 'stop_lat', 'location_type', 'parent_station']],
          stops],
         sort=False,
         ignore_index=True)
stops.to_csv('data/gtfs/stops.txt', sep=',', index=False)
stops.head()

,stop_id,stop_name,stop_lon,stop_lat,location_type,parent_station
0,1009283,"Китай-город, вход-выход 9 в северный вестибюль",37.631677,55.757315,2,1009090
1,1009284,"Китай-город, вход-выход 4 в северный вестибюль",37.631205,55.756855,2,1009090
2,1009285,"Китай-город, вход-выход 6 в северный вестибюль",37.631951,55.757236,2,1009090
3,1009286,"Китай-город, вход-выход 14 в южный вестибюль",37.633082,55.753098,2,1009090
4,1009287,"Китай-город, вход-выход 12 в южный вестибюль",37.633559,55.753072,2,1009090


## routes.txt

***route_id***	Идентификатор	Обязат.	Определяет маршрут.


***agency_id***	Идентификатор из поля agency.agency_id	Условно обязат.	Агентство для указанного маршрута. Это поле является обязательным, если в файле agency.txt представлены данные о маршрутах нескольких разных агентств.


***route_short_name***	Текст	Условно обязат.	Краткое название маршрута: используемое пассажирами абстрактное обозначение (например, "32", 100X, "зеленая линия"), которое не содержит информацию о местах следования маршрута. Необходимо указать значение хотя бы одного из этих полей (route_short_name или route_long_name). При желании можно задать их оба.


***route_long_name***	Текст	Условно обязат.	Полное название маршрута: более описательное, чем у route_short_name, часто с названием пункта назначения или остановки маршрута. Необходимо указать значение хотя бы одного из этих полей (route_short_name или route_long_name). При желании можно задать их оба.

***route_type***	Перечисление	Обязат.	Тип транспорта, используемого на маршруте. Допустимые значения:

0 – трамвай, легкая железная дорога, любые другие системы легкорельсового уличного транспорта в пределах мегаполиса.
1 – метро и любые другие системы подземного железнодорожного транспорта в пределах мегаполиса.
2 – железная дорога для пригородных и междугородних перевозок.
3 – автобусы (внутригородские и междугородние).
4 – паромы (независимо от протяженности маршрутов).
5 – канатные трамваи и системы уличного транспорта с подвагонным приводным тросом, такие как канатные трамваи в Сан-Франциско.
6 – подъемники, подвесные канатные трамваи и другие виды транспорта, в которых кабины, вагоны или кресла удерживаются одним или несколькими тросами.
7 – фуникулеры и любые другие системы железнодорожного транспорта, предназначенные для движения по крутым склонам.
11 – троллейбусы и электробусы, получающие электрический ток через двухпроводную контактную сеть, расположенную над ними.
12 – монорельсовый транспорт, в котором используется один несущий рельс.

In [34]:
routes = pd.read_csv('data/gtfs/routes.txt')
max_route_id = routes.route_id.max()
print(max_route_id)
routes.head(2)

14150253


,route_id,agency_id,route_short_name,route_long_name,route_type,route_desc
0,14150158,1,597,"Метро ""Сокол"" - Набережная Новикова-Прибоя",3,NaN
1,14150132,2,380,Ст. МЦД Дегунино - Пр. Русанова,3,NaN


In [35]:
metro_routes = df.drop_duplicates(subset='Line').reset_index(drop=True)[['Line']]
metro_routes.columns = ['route_long_name']
metro_routes['route_id'] = range(max_route_id+1, max_route_id+1+len(metro_routes))
metro_routes['agency_id'] = 10
metro_routes['route_type'] = 1
routes = pd.concat([metro_routes,
          routes],
         sort=False,
         ignore_index=True)
routes.to_csv('data/gtfs/routes.txt', index=False, sep=',')
routes.head()

,route_long_name,route_id,agency_id,route_type,route_short_name,route_desc
0,Калужско-Рижская линия,14150254,10,1,NaN,NaN
1,Люблинско-Дмитровская линия,14150255,10,1,NaN,NaN
2,Бутовская линия Лёгкого метро,14150256,10,1,NaN,NaN
3,Серпуховско-Тимирязевская линия,14150257,10,1,NaN,NaN
4,Каховская линия,14150258,10,1,NaN,NaN


In [36]:
os.remove(temp_file)

In [37]:
any([])

False